# Simple Timestream Test

Basic notebook to write and read data from AWS Timestream.

In [ ]:
import boto3
import time
from datetime import datetime

print("Initializing Timestream clients...")

In [ ]:
# Configuration
region = 'us-east-1'
database_name = 'SuperAppDB'
table_name = 'UEReports'

# Create Timestream clients
ts_write = boto3.client('timestream-write', region_name=region)
ts_query = boto3.client('timestream-query', region_name=region)

print(f"✓ Timestream clients created")
print(f"  Region: {region}")
print(f"  Database: {database_name}")
print(f"  Table: {table_name}")

## Step 1: Create Database and Table (if needed)

In [ ]:
# Create database
try:
    ts_write.create_database(DatabaseName=database_name)
    print(f"✓ Database '{database_name}' created")
except ts_write.exceptions.ConflictException:
    print(f"Database '{database_name}' already exists")
except Exception as e:
    print(f"Error creating database: {e}")

In [ ]:
# Create table
try:
    ts_write.create_table(
        DatabaseName=database_name,
        TableName=table_name,
        RetentionProperties={
            'MemoryStoreRetentionPeriodInHours': 24,
            'MagneticStoreRetentionPeriodInDays': 7
        }
    )
    print(f"✓ Table '{table_name}' created")
    time.sleep(10)  # Wait for table to be active
except ts_write.exceptions.ConflictException:
    print(f"Table '{table_name}' already exists")
except Exception as e:
    print(f"Error creating table: {e}")

## Step 2: Write Sample Records

In [ ]:
# Prepare sample records
current_time = str(int(time.time() * 1000))  # Current time in milliseconds

records = [
    {
        'Time': current_time,
        'Dimensions': [
            {'Name': 'device_id', 'Value': 'device-001'},
            {'Name': 'location', 'Value': 'datacenter-1'},
            {'Name': 'region', 'Value': 'us-east-1'}
        ],
        'MeasureName': 'cpu_usage',
        'MeasureValue': '75.5',
        'MeasureValueType': 'DOUBLE'
    },
    {
        'Time': current_time,
        'Dimensions': [
            {'Name': 'device_id', 'Value': 'device-002'},
            {'Name': 'location', 'Value': 'datacenter-1'},
            {'Name': 'region', 'Value': 'us-east-1'}
        ],
        'MeasureName': 'cpu_usage',
        'MeasureValue': '82.3',
        'MeasureValueType': 'DOUBLE'
    },
    {
        'Time': current_time,
        'Dimensions': [
            {'Name': 'device_id', 'Value': 'device-003'},
            {'Name': 'location', 'Value': 'datacenter-2'},
            {'Name': 'region', 'Value': 'us-west-2'}
        ],
        'MeasureName': 'memory_usage',
        'MeasureValue': '64.8',
        'MeasureValueType': 'DOUBLE'
    }
]

print(f"Prepared {len(records)} records to write")
print(f"Timestamp: {datetime.fromtimestamp(int(current_time)/1000)}")

In [ ]:
# Write records to Timestream
try:
    result = ts_write.write_records(
        DatabaseName=database_name,
        TableName=table_name,
        Records=records
    )
    print("✓ Records written successfully")
    print(f"  Records processed: {result['RecordsIngested']['Total']}")
except Exception as e:
    print(f"Error writing records: {e}")

## Step 3: Query Records

In [ ]:
# Query all recent records
query = f"""
SELECT * 
FROM "{database_name}"."{table_name}"
WHERE time > ago(1h)
ORDER BY time DESC
LIMIT 10
"""

print("Executing query:")
print(query)
print("\n" + "="*60)

In [ ]:
# Execute query
try:
    response = ts_query.query(QueryString=query)
    
    # Display results
    print("QUERY RESULTS:")
    print("="*60)
    
    if len(response['Rows']) == 0:
        print("No records found")
    else:
        # Print column headers
        columns = [col['Name'] for col in response['ColumnInfo']]
        print(" | ".join(columns))
        print("-" * 60)
        
        # Print rows
        for row in response['Rows']:
            values = []
            for data in row['Data']:
                if 'ScalarValue' in data:
                    values.append(data['ScalarValue'])
                else:
                    values.append('NULL')
            print(" | ".join(values))
        
        print("\n" + "="*60)
        print(f"Total records: {len(response['Rows'])}")
        
except Exception as e:
    print(f"Error querying data: {e}")

## Step 4: Query with Aggregation

In [ ]:
# Aggregate query - average CPU usage by device
agg_query = f"""
SELECT 
    device_id,
    AVG(CAST(measure_value::double AS DOUBLE)) as avg_value,
    COUNT(*) as record_count
FROM "{database_name}"."{table_name}"
WHERE measure_name = 'cpu_usage'
  AND time > ago(1h)
GROUP BY device_id
ORDER BY avg_value DESC
"""

print("Aggregate Query:")
print(agg_query)
print("\n" + "="*60)

In [ ]:
# Execute aggregate query
try:
    response = ts_query.query(QueryString=agg_query)
    
    print("AGGREGATION RESULTS:")
    print("="*60)
    
    if len(response['Rows']) == 0:
        print("No CPU usage records found")
    else:
        # Print headers
        print("Device ID | Average CPU | Record Count")
        print("-" * 60)
        
        # Print results
        for row in response['Rows']:
            device = row['Data'][0]['ScalarValue']
            avg_cpu = float(row['Data'][1]['ScalarValue'])
            count = row['Data'][2]['ScalarValue']
            print(f"{device:15} | {avg_cpu:11.2f} | {count}")
        
        print("="*60)
        
except Exception as e:
    print(f"Error querying data: {e}")

## Test Complete

If you see data above, Timestream integration is working correctly!